In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
import pickle
import numpy as np
import os

In [3]:
file = open("C:/Users/Yaniv/Desktop/cnn/The Adventures of Sherlock Holme.txt", "r", encoding = "utf8")

# store file in list
lines = []
for i in file:
    lines.append(i)

# Convert list to string
data = ""
for i in lines:
  data = ' '. join(lines) 

#replace unnecessary stuff with space
data = data.replace('\n', '').replace('\r', '').replace('\ufeff', '').replace('“','').replace('”','')  #new line, carriage return, unicode character --> replace by space

#remove unnecessary spaces 
data = data.split()
data = ' '.join(data)
data[:500]

'The Project Gutenberg eBook of The Adventures of Sherlock Holmes, by Arthur Conan Doyle This eBook is for the use of anyone anywhere in the United States and most other parts of the world at no cost and with almost no restrictions whatsoever. You may copy it, give it away or re-use it under the terms of the Project Gutenberg License included with this eBook or online at www.gutenberg.org. If you are not located in the United States, you will have to check the laws of the country where you are lo'

In [4]:
len(data)

20100

In [5]:
tokenizer = Tokenizer()
t=tokenizer.fit_on_texts([data])
print(t)

None


In [7]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts([data])

# saving the tokenizer for predict function
pickle.dump(tokenizer, open('token.pkl', 'wb'))

sequence_data = tokenizer.texts_to_sequences([data])[0]
sequence_data[:15]

[1, 114, 91, 83, 2, 1, 115, 2, 92, 18, 35, 138, 139, 140, 29]

In [8]:
len(sequence_data)


3707

In [9]:
vocab_size = len(tokenizer.word_index) + 1
print(vocab_size)

1224


In [10]:
sequences = []

for i in range(3, len(sequence_data)):
    words = sequence_data[i-3:i+1]
    sequences.append(words)
    
print("The Length of sequences are: ", len(sequences))
sequences = np.array(sequences)
sequences[:10]

The Length of sequences are:  3704


array([[  1, 114,  91,  83],
       [114,  91,  83,   2],
       [ 91,  83,   2,   1],
       [ 83,   2,   1, 115],
       [  2,   1, 115,   2],
       [  1, 115,   2,  92],
       [115,   2,  92,  18],
       [  2,  92,  18,  35],
       [ 92,  18,  35, 138],
       [ 18,  35, 138, 139]])

In [11]:
X = []
y = []

for i in sequences:
    X.append(i[0:3])
    y.append(i[3])
    
X = np.array(X)
y = np.array(y)

In [12]:
print("Data: ", X[:10])
print("Response: ", y[:10])

Data:  [[  1 114  91]
 [114  91  83]
 [ 91  83   2]
 [ 83   2   1]
 [  2   1 115]
 [  1 115   2]
 [115   2  92]
 [  2  92  18]
 [ 92  18  35]
 [ 18  35 138]]
Response:  [ 83   2   1 115   2  92  18  35 138 139]


In [13]:
y = to_categorical(y, num_classes=vocab_size)
y[:5]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.]], dtype=float32)

In [14]:
model = Sequential()
model.add(Embedding(vocab_size, 10, input_length=3))
model.add(LSTM(1000, return_sequences=True))
model.add(LSTM(1000))
model.add(Dense(1000, activation="relu"))
model.add(Dense(vocab_size, activation="softmax"))

In [15]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 3, 10)             12240     
                                                                 
 lstm (LSTM)                 (None, 3, 1000)           4044000   
                                                                 
 lstm_1 (LSTM)               (None, 1000)              8004000   
                                                                 
 dense (Dense)               (None, 1000)              1001000   
                                                                 
 dense_1 (Dense)             (None, 1224)              1225224   
                                                                 
Total params: 14,286,464
Trainable params: 14,286,464
Non-trainable params: 0
_________________________________________________________________


In [16]:
from tensorflow.keras.callbacks import ModelCheckpoint

checkpoint = ModelCheckpoint("next_words.h5", monitor='loss', verbose=1, save_best_only=True)
model.compile(loss="categorical_crossentropy", optimizer=Adam(learning_rate=0.001))
model.fit(X, y, epochs=50, batch_size=64, callbacks=[checkpoint])

Epoch 1/50
58/58 [==============================] - ETA: 0s - loss: 6.4856
Epoch 1: loss improved from inf to 6.48563, saving model to next_words.h5
58/58 [==============================] - 20s 226ms/step - loss: 6.4856
Epoch 2/50
58/58 [==============================] - ETA: 0s - loss: 6.0864
Epoch 2: loss improved from 6.48563 to 6.08639, saving model to next_words.h5
58/58 [==============================] - 13s 224ms/step - loss: 6.0864
Epoch 3/50
58/58 [==============================] - ETA: 0s - loss: 6.0055
Epoch 3: loss improved from 6.08639 to 6.00552, saving model to next_words.h5
58/58 [==============================] - 13s 222ms/step - loss: 6.0055
Epoch 4/50
58/58 [==============================] - ETA: 0s - loss: 5.9181
Epoch 4: loss improved from 6.00552 to 5.91810, saving model to next_words.h5
58/58 [==============================] - 13s 221ms/step - loss: 5.9181
Epoch 5/50
58/58 [==============================] - ETA: 0s - loss: 5.7582
Epoch 5: loss improved from 5.918

58/58 [==============================] - 13s 228ms/step - loss: 0.7456
Epoch 38/50
58/58 [==============================] - ETA: 0s - loss: 0.6781
Epoch 38: loss improved from 0.74564 to 0.67812, saving model to next_words.h5
58/58 [==============================] - 13s 226ms/step - loss: 0.6781
Epoch 39/50
58/58 [==============================] - ETA: 0s - loss: 0.5651
Epoch 39: loss improved from 0.67812 to 0.56514, saving model to next_words.h5
58/58 [==============================] - 13s 225ms/step - loss: 0.5651
Epoch 40/50
58/58 [==============================] - ETA: 0s - loss: 0.4999
Epoch 40: loss improved from 0.56514 to 0.49993, saving model to next_words.h5
58/58 [==============================] - 13s 229ms/step - loss: 0.4999
Epoch 41/50
58/58 [==============================] - ETA: 0s - loss: 0.4545
Epoch 41: loss improved from 0.49993 to 0.45455, saving model to next_words.h5
58/58 [==============================] - 13s 226ms/step - loss: 0.4545
Epoch 42/50
58/58 [======

In [68]:
import pickle
with open('NLP.pickle','wb') as f:
    pickle.dump("next_words.h5",f)
    

In [69]:
with open('NLPTokenizer.pickle','wb') as f:
    pickle.dump("token.pkl",f)

In [70]:
import json
columns = {
    'data_columns' : [col.lower() for col in 'next_words.h5']
}
with open("NLPmodel.json","w") as f:
    f.write(json.dumps(columns))

In [17]:
from tensorflow.keras.models import load_model
import numpy as np
import pickle

# Load the model and tokenizer
model = load_model('next_words.h5')
tokenizer = pickle.load(open('token.pkl', 'rb'))

In [19]:
#sequence = tokenizer.texts_to_sequences([text])


In [20]:
from tensorflow.keras.models import load_model
import numpy as np
import pickle

# Load the model and tokenizer
model = load_model('next_words.h5')
tokenizer = pickle.load(open('token.pkl', 'rb'))

def Predict_Next_Words(model, tokenizer, text):
  sequence = tokenizer.texts_to_sequences([text])
  print(sequence)
  sequence = np.array(sequence)
  print(sequence)
  preds = np.argmax(model.predict(sequence))
  predicted_word = ""
  
  for key, value in tokenizer.word_index.items():
      if value == preds:
          predicted_word = key
          break
  
  print(predicted_word)
  return predicted_word

In [ ]:
while(True):
  text = input("Enter your line: ")
  
  if text == "0":
      print("Execution completed.....")
      break
  
  else:
      try:
          text = text.split(" ")
          text = text[-3:]
          print(text)
        
          Predict_Next_Words(model, tokenizer, text)
          
      except Exception as e:
        print("Error occurred: ",e)
        continue

Enter your line: The Robert
['The', 'Robert']
[[1]]
[[1]]
1/1 [==============================] - 1s 844ms/step
august
